In [3]:
import sklearn
import numpy as np

We'll first look at linear regression. We can use the LinearRegression() object from sklearn.linear_model

In [4]:
# create data
pizza_data = np.array([[2100,  800],
                       [2500,  850],
                       [1800,  760],
                       [2000,  800],
                       [2300,  810]])

pizza_prices = np.array([10.99, 12.5 ,  9.99, 10.99, 11.99])

In [5]:
from sklearn import linear_model

# create instance of LinearRegression object
reg = linear_model.LinearRegression()
# fit the linear regression objection
reg.fit(X = pizza_data, y = pizza_prices) # no return type, but now the object has intercept and coefficient parameters


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now we can use this fitted model to make predictions. It already has the necessary coefficients and intercept
instantiated as attributes

In [8]:
# new pizza data
new_pizzas = np.array([[2000,  820],
                       [2200,  830]])

price_predicts = reg.predict(new_pizzas)
print("\nPredictions for new inputs: ")
print(price_predicts)
print("\n")

# get coefficients, intercept of model:
print('Coefficients: {}\n'.format(repr(reg.coef_)))
print('Intercept: {}\n'.format(reg.intercept_))

# get R^2 of the data:
r2 = reg.score(pizza_data, pizza_prices)
print('R2: {}\n'.format(r2))


Predictions for new inputs: 
[10.86599206 11.55111111]


Coefficients: array([0.00330913, 0.00232937])

Intercept: 2.3376587301587346

R2: 0.9758349388652625



One modification of linear regression is ridge regression. It's similar to linear regression, but you want to push the coefficients to be as small as possible

With ridge regression, the goal is to find coefficients (and hypertune for $\alpha$) that minimize the following:

$\alpha ||\omega||^2_{2} + \sum\limits_{i = 1}^{n} (x_i * \omega - y_i)^2$ 

Where $\omega$ represents the weights, and $\alpha ||\omega||^2_{2} represents the penalty term. As the weights increase, the penalty term increases, so this will tend to drive down the weights

We can implement a Ridge regression similar to a regular regression, except now we need to specify $\alpha$

In [9]:
# ridge regression
ridge_reg = linear_model.Ridge(alpha = 0.1)
ridge_reg.fit(pizza_data, pizza_prices)
print('Coefficients: {}\n'.format(repr(ridge_reg.coef_)))
print('Intercept: {}\n'.format(ridge_reg.intercept_))
r2_ridge = ridge_reg.score(pizza_data, pizza_prices)
print('R2: {}\n'.format(r2_ridge))

Coefficients: array([0.00330919, 0.0023288 ])

Intercept: 2.337978289647138

R2: 0.9758349388362841



We can modify the ridge regression by using cross-validation to find the optimal alpha. Sklearn includes an object, RidgeCV, that combines ridge regression and cross-validation. 

In [10]:
alphas = [0.1, 0.2, 0.3]
ridge_reg_cv = linear_model.RidgeCV(alphas=alphas)
ridge_reg_cv.fit(pizza_data, pizza_prices)
print('Coefficients: {}\n'.format(repr(ridge_reg_cv.coef_)))
print('Intercept: {}\n'.format(ridge_reg_cv.intercept_))
print('Chosen alpha: {}\n'.format(ridge_reg_cv.alpha_))

Coefficients: array([0.00330932, 0.00232767])

Intercept: 2.3386168534481815

Chosen alpha: 0.3



Lasso regression is an alternative to ridge regression that drives some coefficients to zero. It uses the L1 norm for its objective function:

$\alpha ||\omega||_{1} + \sum\limits_{i = 1}^{n} (x_i * \omega - y_i)^2$ 

LASSO regularization tends to prefer linear models with fewer parameter values. This means that it will likely zero-out some of the weight coefficients. This reduces the number of features that the model is actually dependent on (since some of the coefficients will now be 0), which can be beneficial when some features are completely irrelevant or duplicates of other features.

In [12]:
"""
print('Data shape: {}\n'.format(data.shape))
print('Labels shape: {}\n'.format(labels.shape))

from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit(data, labels)
print('Coefficients: {}\n'.format(repr(reg.coef_)))
print('Intercept: {}\n'.format(reg.intercept_))
print('R2: {}\n'.format(reg.score(data, labels)))
"""

"""
Data shape: (150, 4)

Labels shape: (150,)

Coefficients: array([ 0.        , -0.        ,  0.40830957,  0.        ])

Intercept: -0.534699558318563

R2: 0.895831189504504
"""

# Notice that the majority of weights are 0, due to the lasso regression's preference for sparsity
# there is also a cross-validated version of lasso, LassoCV

'\nData shape: (150, 4)\n\nLabels shape: (150,)\n\nCoefficients: array([ 0.        , -0.        ,  0.40830957,  0.        ])\n\nIntercept: -0.534699558318563\n\nR2: 0.895831189504504\n'

Now, we turn to Bayesian regression methods

We can use Bayesian methods to assume that our model parameters/hyperparameters come from a distribution. 

In the case of Bayesian ridge regression, we can assume that $\alpha$ is our penalty hyperparameter, and this comes from an underlying distribution (default: gamma), while $\lambda$ is our weight hyperparameter, which acts on the precision of the weights (the smaller the value for $\lambda$, the greater the variance between individual weights

Both hyperparameters are assumed to come from a gamma distribution, and a gamma distribution has two parameters (shape and scale) that are both $10^{-6}$ by default

When finding the optimal weight settings of a Bayesian ridge regression model for an input dataset, we also concurrently optimize the α and λ hyperparameters based on their prior distributions and the input data.

This can all be done with the BayesianRidge object (part of the linear_model module). Like all the previous regression objects, this one can be initialized with no required arguments.

In [13]:
"""
# predefined dataset from previous chapter
print('Data shape: {}\n'.format(data.shape))
print('Labels shape: {}\n'.format(labels.shape))

from sklearn import linear_model
reg = linear_model.BayesianRidge()
reg.fit(data, labels)
print('Coefficients: {}\n'.format(repr(reg.coef_)))
print('Intercept: {}\n'.format(reg.intercept_))
print('R2: {}\n'.format(reg.score(data, labels)))
print('Alpha: {}\n'.format(reg.alpha_))
print('Lambda: {}\n'.format(reg.lambda_))
"""

"""
Data shape: (150, 4)

Labels shape: (150,)

Coefficients: array([-0.11174619, -0.03900476,  0.24330537,  0.57343721])

Intercept: 0.17022693722601356

R2: 0.9303454031271241

Alpha: 20.983865171760993

Lambda: 9.545491382343116

"""


'\nData shape: (150, 4)\n\nLabels shape: (150,)\n\nCoefficients: array([-0.11174619, -0.03900476,  0.24330537,  0.57343721])\n\nIntercept: 0.17022693722601356\n\nR2: 0.9303454031271241\n\nAlpha: 20.983865171760993\n\nLambda: 9.545491382343116\n\n'

Now here we have the sklearn implementation of logistic regression. For multiclass logistic regression, we initialize the LogisticRegression() object with the multi_class argument. The default is 'ovr', which is a "One-Vs-Rest" strategy (picks the argmax out of the probabilities). 

In multiclass classification, we want to use the 'multinomial' strategy. Note that to use the 'multinomial' strategy, we need to choose a proper solver (see below for details on solvers). In this case, we choose 'lbfgs'.

By default, the logistic regression is regularized through the L2 norm of weights. We can manually specify whether to use the L1 or L2 norm with the penalty keyword argument, by setting it as either 'l1' or 'l2'.

We can choose a particular solver using the solver keyword argument. The default solver is currently 'liblinear' (although it will change to 'lbfgs' in future version). For the 'newton-cg', 'sag', and 'lbfgs' solvers, we can also set the maximum number of iterations the solver takes until the model's weights converge using the max_iter keyword argument. Since the default max_iter value is 100, we may want to let the solver run for a higher number of iterations in certain applications.

Like the ridge and LASSO regression models, the logistic regression model comes with a cross-validated version in scikit-learn. The cross-validated logistic regression object, LogisticRegressionCV, is initialized and used in the same way as the regular LogisticRegression object.

In [ ]:
# example of fitting logistic regression
reg = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)

# example of fitting with multiclass specification (here, we have to include the solver)
reg = linear_model.LogisticRegression(solver='lbfgs',
                                      multi_class='multinomial', 
                                      max_iter = 1000)

# fitting cross-validated version:
reg = linear_model.LogisticRegressionCV(solver='multinomial', 
                                        max_iter=1000)

In lieu of regressions, we can instead work with trees. Decision trees are binary trees. The leaves of the decision tree determine the class label to predict (in classification) or the real number value to predict (in regression).

We can specify whether our tree is designed for regression or classification. In scikit-learn, we implement classification decision trees with the DecisionTreeClassifier object, and regression trees with the DecisionTreeRegressor object. Both objects are part of the tree module in scikit-learn.

In [ ]:
from sklearn import tree

"""
clf_tree1 = tree.DecisionTreeClassifier()
reg_tree1 = tree.DecisionTreeRegressor()
clf_tree2 = tree.DecisionTreeClassifier(
  max_depth=8)  # max depth of 8
reg_tree2 = tree.DecisionTreeRegressor(
  max_depth=5)  # max depth of 5

# predefined dataset
print('Data shape: {}\n'.format(data.shape))
# Binary labels
print('Labels:\n{}\n'.format(repr(labels)))
clf_tree1.fit(data, labels)
"""

"""
from sklearn import tree
d_tree = tree.DecisionTreeRegressor(max_depth = 5)
d_tree.fit(data, labels)
"""

# by default, there is no max depth. But, trees that are too deep can overfit. So, it's good to have a max depth

How do we determine the features to include at each node of the tree? Since scikit-learn uses the CART algorithm, we use Gini Impurity (for classification), MSE (mean squared error) (for regression), and MAE (mean absolute error) (for regression) to decide on the best feature at each node.

Splitting the data into training and test sets:

In general, the testing set is around 10-30% of the original dataset, while the training set makes up the remaining 70-90%. Increasing the size of the training set will give more data for the model to be fitted on, which can increase the model's performance. However, because this decreases the size of the testing set, there's a higher chance that the testing set may not be representative of the original dataset (which can lead to inaccurate evaluation).

The scikit-learn library provides a nice utility function, called train_test_split (which is part of the model_selection module) that handles the dataset splitting for us.

Note that the train_test_split function randomly shuffles the dataset and corresponding labels prior to splitting. This is good practice to remove any systematic orderings in the dataset, which could potentially impact the model into training on the orderings rather than the actual data.

The default size of the testing set is 25% of the original dataset. We can use the test_size keyword argument to manually specify the proportion of the original dataset that will go into the testing set.

In [15]:
# initialize data
data = np.array([
  [10.2 ,  0.5 ],
  [ 8.7 ,  0.9 ],
  [ 9.3 ,  0.8 ],
  [10.1 ,  0.4 ],
  [ 9.5 ,  0.77],
  [ 9.1 ,  0.68],
  [ 7.7 ,  0.9 ],
  [ 8.3 ,  0.8 ]])
labels = np.array(
  [1.4, 1.2, 1.6, 1.5, 1.6, 1.3, 1.1, 1.2])

In [17]:
split_dataset

[array([[ 7.7 ,  0.9 ],
        [ 8.7 ,  0.9 ],
        [ 9.1 ,  0.68],
        [ 9.5 ,  0.77],
        [10.1 ,  0.4 ],
        [ 9.3 ,  0.8 ]]), array([[10.2,  0.5],
        [ 8.3,  0.8]]), array([1.1, 1.2, 1.3, 1.6, 1.5, 1.6]), array([1.4, 1.2])]

In [19]:
# get train/test split

from sklearn.model_selection import train_test_split
# output has split data, in 4 sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = .2) 
print('{}\n'.format(repr(train_data)))
print('{}\n'.format(repr(train_labels)))
print('{}\n'.format(repr(test_data)))
print('{}\n'.format(repr(test_labels)))

array([[ 9.5 ,  0.77],
       [10.2 ,  0.5 ],
       [ 8.7 ,  0.9 ],
       [ 7.7 ,  0.9 ],
       [ 8.3 ,  0.8 ],
       [ 9.1 ,  0.68]])

array([1.6, 1.4, 1.2, 1.1, 1.2, 1.3])

array([[10.1,  0.4],
       [ 9.3,  0.8]])

array([1.5, 1.6])



Now, let's evaluate our results using cross-validation:

Cross-validation creates synthetic validation sets by partitioning the training set into multiple smaller subsets. One of the most common algorithms for cross-validation, K-Fold CV, partitions the training set into k approximately equal sized subsets (referred to as folds). There are k "rounds" of the algorithm, and each "round" chooses one of the k subsets for the validation set (a different subset is chosen each round), while the remaining k - 1 subsets are aggregated into the round's training set and used to train the model.

So, let's say that k = 5. Then, the data is divided into 5 folds (so each observation belongs to 1 of 5 folds). In the first run of the algorithm, we take data in folds 2-5 as the training set and data in fold 1 as the test set. In the second run of the algorithm, we take data in folds 1, 3-5 as the training set and data in fold 2 as the test set. We continue this until we've used each of the 5 folds as a test set. 

Each round of the K-Fold algorithm, the model is trained on that round's training set (the combined training folds) and then evaluated on the single validation fold. The evaluation metric depends on the model. For classification models, this is usually classification accuracy on the validation set. For regression models, this can either be the model's mean squared error, mean absolute error, or R2 value on the validation set.

We can implement k-fold cross validation (with scores/metrics) using sklearn's cross_val_score() function. To use the function, we need to input (1) an instantiated model (E.g., sklearn.LogisticRegression()), (2) input data X, (3) labels y, (4) cv = #, indicating # of folds

In [20]:
""" k-fold cross validation, with classification
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
clf = linear_model.LogisticRegression()
# Predefined data and labels
cv_score = cross_val_score(clf, data, labels, cv=3)  # k = 3

print('{}\n'.format(repr(cv_score))) # here, the evaluation metric, by default, is accuracy, since it's classification
"""

"""
OUTPUT:
array([0.93684211, 0.96842105, 0.94179894])
"""


""" k-fold cross validation, with regression
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
reg = linear_model.LinearRegression()
# Predefined data and labels
cv_score = cross_val_score(reg, data, labels, cv=4)  # k = 4

print('{}\n'.format(repr(cv_score))) # evaluation metric is R^2
"""

"""
OUTPUT: 
array([0.37548118, 0.49022643, 0.52061242, 0.54864085])
"""



'\nOUTPUT: \narray([0.37548118, 0.49022643, 0.52061242, 0.54864085])\n'

Note that we don't call fit with the model prior to using cross_val_score. This is because the cross_val_score function will use fit for training the model each round.

For classification, k-fold cross validation will attempt to make sure that each fold will contain approximately the same class distribution as the original dataset (if it doesn't do this, then the class imbalance can lead to poor accuracy results). The cross_val_score function will apply a special form of the K-Fold algorithm called stratified K-Fold

We can also apply cross-validation to trees. We can, for example, tinker with the maximum depth of the tree. K-Fold cross-validation gives an accurate measurement of how good the decision tree is for the dataset. We can use K-Fold cross-validation with different values of the max_depth hyperparameter and see which one gives the best cross-validation scores. We can use the cv_decision_tree function to do so:

In [ ]:
"""
is_clf = True  # for classification
for depth in range(3, 8):  # loop through all trees
    # Predefined data and labels
    
    # we perform cross-validation on a given set of hyperparameters. We use the same
    # hyperparameters for each fold of the decision tree, so that we can understand the nodes
    # that come up with the tree. 
    scores = cv_decision_tree(
        is_clf, data, labels, depth, 5)  # k = 5
    mean = scores.mean()  # Mean acc across folds
    std_2 = 2 * scores.std()  # 2 std devs
    print('95% C.I. for depth {}: {} +/- {:.2f}\n'.format(depth, mean, std_2))
    
    # however, for each iteration of the loop we change the value of the hyperparameter
    # we use the loop for hyperparameter tuning, and we use cross-validation to evaluate 
    # the result of each possible value of the hyperparameter
    
# In the above code, we use the cv_decision_tree function to apply 5-Fold 
# cross-validation to a classification decision tree. We tune its maximum 
# depth hyperparameter across depths of 3, 4, 5, 6, and 7. For each max_depth 
# value, we print the 95% confidence interval for the cross-validated scores across the 5 folds.
"""

To evaluate the results of our models, we can use the 'metrics' module in sklearn. Each of the evaluation functions takes in the actual testing labels as the first argument and the predictions as the second argument.

In [22]:
"""
REGRESSION:

reg = tree.DecisionTreeRegressor()
# predefined train and test sets
reg.fit(train_data, train_labels)
predictions = reg.predict(test_data)

from sklearn import metrics
r2 = metrics.r2_score(test_labels, predictions)
print('R2: {}\n'.format(r2))
mse = metrics.mean_squared_error(
  test_labels, predictions)
print('MSE: {}\n'.format(mse))
mae = metrics.mean_absolute_error(
  test_labels, predictions)
print('MAE: {}\n'.format(mae))
"""

"""
CLASSIFICATION:

clf = tree.DecisionTreeClassifier()
# predefined train and test sets
clf.fit(train_data, train_labels)
predictions = clf.predict(test_data)

from sklearn import metrics
acc = metrics.accuracy_score(test_labels, predictions)
print('Accuracy: {}\n'.format(acc))
"""

"\nCLASSIFICATION:\n\nclf = tree.DecisionTreeClassifier()\n# predefined train and test sets\nclf.fit(train_data, train_labels)\npredictions = clf.predict(test_data)\n\nfrom sklearn import metrics\nacc = metrics.accuracy_score(test_labels, predictions)\nprint('Accuracy: {}\n'.format(acc))\n"

We can use sklearn to perform grid search, in which we take each combination of a set of hyperparameters, apply cross-validation, and see which one gives the best results. For the grid search cross-validation, we specify possible values for each hyperparameter, and then the search will go through each possible combination of the hyperparameters and return the model with the best combination.

Python implements this through the GridSearchCV() object, which is in the model_selection module. 

In [23]:
"""
reg = linear_model.BayesianRidge()
params = {
  'alpha_1':[0.1,0.2,0.3],
  'alpha_2':[0.1,0.2,0.3]
}
reg_cv = GridSearchCV(reg, params, cv=5, iid=False)
# predefined train and test sets
reg_cv.fit(train_data, train_labels)
print(reg_cv.best_params_)
"""

"""
OUTPUT: 
{'alpha_2': 0.1, 'alpha_1': 0.3}
"""

"\nreg = linear_model.BayesianRidge()\nparams = {\n  'alpha_1':[0.1,0.2,0.3],\n  'alpha_2':[0.1,0.2,0.3]\n}\nreg_cv = GridSearchCV(reg, params, cv=5, iid=False)\n# predefined train and test sets\nreg_cv.fit(train_data, train_labels)\nprint(reg_cv.best_params_)\n"

The cv keyword argument represents the number of folds used in the K-Fold cross-validation for grid search. The iid keyword argument relates to how the cross-validation score is calculated. We use False to match the standard definition of cross-validation. Note that in later updates of scikit-learn, the iid argument will be removed from GridSearchCV.

Since exhaustive grid search performs cross-validation on each possible hyperparameter value combination, it can be incredibly slow for larger datasets. It should only be used if the dataset is reasonably small and it is important to choose the best hyperparameter combination.

Grid search iterates through each combination of the hyperparameters and does cross-validation. Because the number of combinations can be incredibly high (e.g., for three hyperparameters with 4 possible values each, that's 64 combinations), doing this on a large dataset will probably be pretty slow